# 뉴스 편향성 분석 모델

이 노트북은 뉴스 기사의 민주당과 국힘에 대한 편향성을 분석하는 딥러닝 모델을 구현합니다.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. 데이터 로드 및 전처리

In [3]:
# 베이스라인 결과 로드
baseline_results = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/baseline_results/baseline_results.csv').iloc[0]

In [4]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/정당_관점_라벨링_최종.csv')  # 파일 경로는 실제 경로에 맞게 수정해주세요

# 정당 레이블 매핑
party_mapping = {'국민의힘': 0, '민주당': 1, '그외': 2}

# 정당 레이블 변환
df['party_label'] = df['party'].map(party_mapping)

# NaN 값 처리
df = df.dropna(subset=['title_cleaned', 'content_cleaned', 'party_label'])

# 제목과 본문 결합
df['text'] = df['title_cleaned'] + ' ' + df['content_cleaned']

print(f"전체 데이터 수: {len(df)}")
print("\n정당별 기사 수:")
print(df['party'].value_counts())
print("\n정당 레이블 분포:")
print(df['party_label'].value_counts())

전체 데이터 수: 1000

정당별 기사 수:
party
그외      420
민주당     314
국민의힘    266
Name: count, dtype: int64

정당 레이블 분포:
party_label
2    420
1    314
0    266
Name: count, dtype: int64


## 2. 데이터셋 클래스 정의

In [5]:
class NewsDataset(Dataset):
    def __init__(self, texts, party_labels, tokenizer, max_length=512):
        self.texts = texts
        self.party_labels = party_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # 텐서 차원 조정
        input_ids = encoding['input_ids'].squeeze(0)  # [max_length]
        attention_mask = encoding['attention_mask'].squeeze(0)  # [max_length]

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'party_label': torch.tensor(self.party_labels[idx], dtype=torch.long)
        }

## 3. 모델 정의

In [6]:
class NewsBiasModel(torch.nn.Module):
    def __init__(self, model_name, num_party_labels=3, class_weights=None,
                 dropout_rate=0.2, hidden_size=256):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))

        # 드롭아웃 레이어
        self.dropout = torch.nn.Dropout(dropout_rate)

        # 특성 추출 레이어 (단순화)
        hidden_size_bert = self.bert.config.hidden_size
        self.feature_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_bert, hidden_size),
            torch.nn.LayerNorm(hidden_size),  # Layer Normalization만 추가
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate)
        )

        # 정당 분류기 (단순화)
        self.party_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, num_party_labels)
        )

        self._init_weights()

    def _init_weights(self):
        """모델 가중치 초기화"""
        for module in [self.feature_layer, self.party_classifier]:
            for m in module.modules():
                if isinstance(m, torch.nn.Linear):
                    torch.nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        torch.nn.init.zeros_(m.bias)

    def forward(self, input_ids, attention_mask, party_label=None):
        # BERT 출력
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰

        # 특성 추출
        features = self.dropout(pooled_output)
        features = self.feature_layer(features)

        # 정당 예측
        party_logits = self.party_classifier(features)

        if party_label is not None:
            # 손실 계산
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            party_loss = loss_fct(party_logits, party_label)
            return {'loss': party_loss, 'party_logits': party_logits}

        return {'party_logits': party_logits}

In [7]:
class MultiTaskNewsModel(torch.nn.Module):
    def __init__(self, model_name, num_stance_labels=3, num_sentiment_labels=3, mask_token_id=50264, class_weights=None):
        super().__init__()
        # 기본 모델 로드
        self.bert = AutoModel.from_pretrained(model_name)
        self.mask_token_id = mask_token_id
        self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))

        # 드롭아웃 레이어
        self.dropout = torch.nn.Dropout(0.2)

        # 공통 특성 추출 레이어
        hidden_size = self.bert.config.hidden_size
        self.shared_layer = torch.nn.Linear(hidden_size, 256)
        self.activation = torch.nn.ReLU()

        # 스탠스 분류기
        self.stance_classifier = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(128, num_stance_labels)
        )

        # 감성 분류기
        self.sentiment_classifier = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(128, num_sentiment_labels)
        )

        # 감성 손실 계산용 vocab projection 레이어
        self.sentiment_vocab_projection = torch.nn.Linear(hidden_size, self.bert.config.vocab_size)

        # 모델 가중치 초기화
        self._init_weights()

    def _init_weights(self):
        """모델 가중치 초기화"""
        for module in [self.shared_layer, self.stance_classifier, self.sentiment_classifier]:
            for m in module.modules():
                if isinstance(m, torch.nn.Linear):
                    torch.nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        torch.nn.init.zeros_(m.bias)

    def forward(self, input_ids, attention_mask, stance_label=None):
        # BERT 출력
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰

        # 공통 특성 추출
        shared_features = self.dropout(pooled_output)
        shared_features = self.shared_layer(shared_features)
        shared_features = self.activation(shared_features)

        # 스탠스 예측
        stance_logits = self.stance_classifier(shared_features)

        # 감성 예측
        sentiment_logits = self.sentiment_classifier(shared_features)

        if stance_label is not None:
            # 스탠스 손실 계산
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            stance_loss = loss_fct(stance_logits, stance_label)

            # 감성 분석 손실 계산
            sentiment_loss = self._compute_sentiment_loss(sentiment_logits, input_ids, attention_mask)

            # 총 손실 계산
            total_loss = stance_loss + 0.5 * sentiment_loss

            return {
                'loss': total_loss,
                'stance_logits': stance_logits,
                'sentiment_logits': sentiment_logits
            }

        return {
            'stance_logits': stance_logits,
            'sentiment_logits': sentiment_logits
        }

    def _compute_sentiment_loss(self, sentiment_logits, input_ids, attention_mask):
        """단순화된 감성 분석 손실 계산"""
        try:
            # 마스킹된 입력 생성
            masked_input_ids = input_ids.clone()
            mask_prob = 0.15

            # 마스킹 마스크 생성 (패딩 토큰 제외)
            mask_mask = torch.rand_like(input_ids.float()) < mask_prob
            mask_mask = mask_mask & (attention_mask == 1)

            if not mask_mask.any():
                return torch.tensor(0.0, device=input_ids.device)

            # 마스킹 적용
            mask_token_id = self.mask_token_id
            masked_input_ids[mask_mask] = mask_token_id

            # 마스킹된 입력에 대한 예측
            # 입력 텐서의 차원 확인 및 조정
            if len(masked_input_ids.shape) == 1:
                masked_input_ids = masked_input_ids.unsqueeze(0)
            if len(attention_mask.shape) == 1:
                attention_mask = attention_mask.unsqueeze(0)

            # BERT 모델에 입력
            masked_outputs = self.bert(
                input_ids=masked_input_ids,
                attention_mask=attention_mask,
                return_dict=True
            )

            # 마스킹된 위치의 로짓 추출
            last_hidden_state = masked_outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
            masked_positions = mask_mask.nonzero(as_tuple=True)  # (batch_indices, seq_indices)

            # 마스킹된 위치의 로짓만 추출
            masked_logits = last_hidden_state[masked_positions]  # [num_masked, hidden_size]

            # 원본 입력과의 차이를 손실로 사용
            target_ids = input_ids[mask_mask]  # [num_masked]

            # 로짓을 어휘 크기에 맞게 변환
            vocab_size = self.bert.config.vocab_size

            # 로짓 투사 후 CrossEntropyLoss 계산
            masked_logits = self.sentiment_vocab_projection(masked_logits)

            # 라벨 범위 검증
            assert torch.max(target_ids) < vocab_size, f"target_id max {torch.max(target_ids)} >= vocab size {vocab_size}"
            assert torch.min(target_ids) >= 0, f"target_id min {torch.min(target_ids)} < 0"

            # 손실 계산
            loss_fct = torch.nn.CrossEntropyLoss()
            sentiment_loss = loss_fct(masked_logits, target_ids)

            return sentiment_loss

        except Exception as e:
            print(f"감성 분석 손실 계산 중 오류 발생: {str(e)}")
            return torch.tensor(0.0, device=input_ids.device)

## 4. 학습 준비

In [8]:
# 데이터 분할
train_texts, val_texts, train_party_labels, val_party_labels = train_test_split(
    df['text'].values,
    df['party_label'].values,
    test_size=0.1,
    random_state=42
)

In [ ]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

# 데이터셋 생성
train_dataset = NewsDataset(train_texts, train_party_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_party_labels, tokenizer)

In [10]:
label_list = [example['party_label'] for example in train_dataset]
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1, 2]),  # 0: 국민의힘, 1: 민주당, 2: 그외
    y=np.array(label_list)
)
print("클래스 가중치:", class_weights)

클래스 가중치: [1.28205128 1.04166667 0.79365079]


In [ ]:
# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화 및 GPU 이동을 분리
model = NewsBiasModel('klue/roberta-base', class_weights=class_weights)
model = model.to(device)

# 커스텀 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
def compute_metrics(eval_pred):
    party_preds = eval_pred.predictions
    party_labels = eval_pred.label_ids

    party_preds = np.argmax(party_preds, axis=1)

    party_report = classification_report(
        party_labels,
        party_preds,
        target_names=['국민의힘', '민주당', '그외'],
        output_dict=True,
        zero_division=0
    )

    return {
        'party_f1': party_report['weighted avg']['f1-score'],
        'party_accuracy': party_report['accuracy'],
    }

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./news_bias_results',  # 디렉토리 이름 변경
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    weight_decay=0.05,
    logging_dir='./news_bias_logs',  # 로그 디렉토리 이름 변경
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='party_f1',  # 평가 메트릭 이름 변경
    gradient_accumulation_steps=2,  # 그래디언트 누적
    fp16=True,  # 혼합 정밀도 학습
    label_smoothing_factor=0.1,  # 레이블 스무딩
    optim='adamw_torch'  # AdamW 옵티마이저 사용
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.001)]
)

## 5. 학습 및 평가

In [ ]:
# 학습 실행
trainer.train()

# 최종 평가
final_metrics = trainer.evaluate()
print("\n최종 평가 결과:")
print(f"정당 분류 F1 점수: {final_metrics['eval_party_f1']:.4f}")
print(f"정당 분류 정확도: {final_metrics['eval_party_accuracy']:.4f}")

In [ ]:
# 베이스라인과 성능 비교
comparison_results = {
    'Model': ['Baseline', 'News Bias BERT'],  # 모델 이름 변경
    'Party F1': [baseline_results['party_f1'], final_metrics['eval_party_f1']],  # 메트릭 이름 변경
    'Party Accuracy': [baseline_results['party_accuracy'], final_metrics['eval_party_accuracy']],  # 메트릭 이름 변경
}

comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv('./news_bias_results/model_comparison.csv', index=False)  # 저장 경로 변경
comparison_df.head()

In [ ]:
# 성능 향상 시각화
plt.figure(figsize=(12, 6))
metrics = ['Party F1', 'Party Accuracy']  # 메트릭 이름 변경
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, comparison_df.iloc[0, 1:], width, label='Baseline')
plt.bar(x + width/2, comparison_df.iloc[1, 1:], width, label='News Bias BERT')  # 모델 이름 변경

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()

plt.savefig('./news_bias_results/performance_comparison.png')  # 저장 경로 변경
plt.show()

## 6. 하이퍼파라미터 튜닝

In [ ]:
!pip install optuna
!pip install optuna-integration[pytorch_lightning]  # 설치 후 세션을 다시 시작해야 함

In [14]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from sklearn.model_selection import KFold

In [15]:
def objective(trial):
    # 하이퍼파라미터 정의
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-4, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 0.01, 0.05),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.3),
        'hidden_size': trial.suggest_int('hidden_size', 128, 512),
        'num_epochs': trial.suggest_int('num_epochs', 5, 15),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32]),
        'warmup_ratio': trial.suggest_float('warmup_ratio', 0.1, 0.2),
        'label_smoothing': trial.suggest_float('label_smoothing', 0.05, 0.15),
        'gradient_accumulation_steps': trial.suggest_int('gradient_accumulation_steps', 1, 4),
        'max_grad_norm': trial.suggest_float('max_grad_norm', 0.5, 1.0)
    }

    # K-fold 교차 검증
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        # 데이터 분할
        train_texts = df.iloc[train_idx]['text'].values
        val_texts = df.iloc[val_idx]['text'].values
        train_labels = df.iloc[train_idx]['party_label'].values
        val_labels = df.iloc[val_idx]['party_label'].values

        # 데이터셋 생성
        train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
        val_dataset = NewsDataset(val_texts, val_labels, tokenizer)

        # 모델 초기화
        model = NewsBiasModel(
            'klue/roberta-base',
            class_weights=class_weights,
            dropout_rate=params['dropout_rate'],
            hidden_size=params['hidden_size']
        )
        model = model.to(device)

        # 학습 인자 설정
        training_args = TrainingArguments(
            output_dir='./temp_results',
            num_train_epochs=params['num_epochs'],
            per_device_train_batch_size=params['batch_size'],
            per_device_eval_batch_size=params['batch_size'] * 2,
            learning_rate=params['learning_rate'],
            warmup_ratio=params['warmup_ratio'],
            weight_decay=params['weight_decay'],
            logging_dir='./temp_logs',
            logging_steps=100,
            eval_strategy='epoch',
            save_strategy='no',
            load_best_model_at_end=False,
            save_total_limit=0,
            metric_for_best_model='party_f1',
            gradient_accumulation_steps=params['gradient_accumulation_steps'],
            max_grad_norm=params['max_grad_norm'],
            fp16=True,
            label_smoothing_factor=params['label_smoothing'],
            optim='adamw_torch',
            lr_scheduler_type='cosine_with_restarts'
        )

        # 트레이너 초기화
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
            compute_metrics=compute_metrics,
            callbacks=[
                EarlyStoppingCallback(
                    early_stopping_patience=3,
                    early_stopping_threshold=0.001
                )
            ]
        )

        # 학습
        trainer.train()

        # 평가
        metrics = trainer.evaluate()
        scores.append(metrics['eval_party_f1'])

    # 평균 F1 점수 반환
    return np.mean(scores)

In [16]:
# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # 20회 시도

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-06-12 14:08:06,382] A new study created in memory with name: no-name-c6bffc55-db63-4ee2-a3ce-ff7622c5e16d
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Party F1,Party Accuracy
1,No log,1.226161,0.214043,0.385000
2,No log,1.071501,0.402148,0.480000
3,No log,0.884938,0.555275,0.560000
4,No log,0.924683,0.619348,0.620000
5,No log,0.960719,0.623257,0.625000
6,0.899000,0.988218,0.631548,0.625000
7,0.899000,1.129396,0.657741,0.655000
8,0.899000,1.442347,0.581717,0.585000
9,0.899000,1.394198,0.637833,0.635000
10,0.899000,1.572592,0.661042,0.665000


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Party F1,Party Accuracy
1,No log,1.136148,0.162721,0.315000
2,No log,1.009193,0.514044,0.510000
3,No log,1.163869,0.387933,0.425000
4,No log,0.850463,0.631350,0.630000
5,No log,1.137863,0.559450,0.550000
6,0.935000,1.008349,0.643159,0.640000
7,0.935000,1.445064,0.610443,0.615000
8,0.935000,1.256246,0.637991,0.635000
9,0.935000,1.518466,0.637820,0.635000


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


...

Epoch,Training Loss,Validation Loss,Party F1,Party Accuracy
1,No log,1.770058,0.223692,0.395000
2,No log,1.215416,0.170045,0.265000
3,No log,1.127652,0.165844,0.270000
4,No log,1.123219,0.148699,0.265000


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Party F1,Party Accuracy
1,No log,1.133578,0.299158,0.360000
2,No log,1.167860,0.118627,0.275000
3,No log,1.081936,0.373426,0.450000
4,No log,1.065340,0.421058,0.475000
5,No log,1.024153,0.486084,0.490000
6,No log,0.975105,0.569771,0.565000
7,No log,0.945399,0.575444,0.575000
8,1.160000,0.919183,0.577688,0.575000
9,1.160000,0.895640,0.588460,0.585000
10,1.160000,0.891036,0.598700,0.595000


[I 2025-06-12 15:28:12,588] Trial 19 finished with value: 0.39416749004862944 and parameters: {'learning_rate': 3.7115502194854245e-06, 'weight_decay': 0.01325740662356341, 'dropout_rate': 0.2999207600288583, 'hidden_size': 280, 'num_epochs': 11, 'batch_size': 32, 'warmup_ratio': 0.12678713956628979, 'label_smoothing': 0.13247546747408104, 'gradient_accumulation_steps': 2, 'max_grad_norm': 0.972587111976976}. Best is trial 8 with value: 0.6747922887361357.


Best trial:
  Value:  0.6747922887361357
  Params: 
    learning_rate: 3.354100053227975e-05
    weight_decay: 0.025573937515468943
    dropout_rate: 0.22975851341070952
    hidden_size: 317
    num_epochs: 9
    batch_size: 32
    warmup_ratio: 0.11351812674740831
    label_smoothing: 0.06721970339723499
    gradient_accumulation_steps: 3
    max_grad_norm: 0.7154661794845409


In [17]:
# 모든 trial의 결과를 DataFrame으로 저장
trials_df = study.trials_dataframe()
trials_df.to_csv('hyperparameter_trials.csv', index=False)
trials_df.head()

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dropout_rate,params_gradient_accumulation_steps,params_hidden_size,params_label_smoothing,params_learning_rate,params_max_grad_norm,params_num_epochs,params_warmup_ratio,params_weight_decay,state
0,0,0.658802,2025-06-12 14:08:06.384102,2025-06-12 14:13:28.657276,0 days 00:05:22.273174,16,0.154166,3,445,0.149606,0.000023,0.864603,14,0.182478,0.046348,COMPLETE
1,1,0.625883,2025-06-12 14:13:28.658324,2025-06-12 14:16:34.610115,0 days 00:03:05.951791,32,0.120297,4,146,0.070386,0.000031,0.561724,7,0.140119,0.040830,COMPLETE
2,2,0.667703,2025-06-12 14:16:34.610999,2025-06-12 14:21:36.864506,0 days 00:05:02.253507,32,0.240416,2,147,0.117206,0.000097,0.853164,12,0.179071,0.049551,COMPLETE
3,3,0.627772,2025-06-12 14:21:36.865404,2025-06-12 14:26:56.702479,0 days 00:05:19.837075,32,0.100879,3,144,0.055942,0.000007,0.717031,13,0.139950,0.017095,COMPLETE
4,4,0.659353,2025-06-12 14:26:56.703408,2025-06-12 14:31:02.187637,0 days 00:04:05.484229,16,0.114403,3,349,0.076953,0.000028,0.603996,12,0.106539,0.048460,COMPLETE


In [21]:
print(trials_df.sort_values(by='value', ascending=False).iloc[0])

number                                                         8
value                                                   0.674792
datetime_start                        2025-06-12 14:43:51.269477
datetime_complete                     2025-06-12 14:47:50.303693
duration                                  0 days 00:03:59.034216
params_batch_size                                             32
params_dropout_rate                                     0.229759
params_gradient_accumulation_steps                             3
params_hidden_size                                           317
params_label_smoothing                                   0.06722
params_learning_rate                                    0.000034
params_max_grad_norm                                    0.715466
params_num_epochs                                              9
params_warmup_ratio                                     0.113518
params_weight_decay                                     0.025574
state                    

In [18]:
# 최적의 하이퍼파라미터로 모델 학습
best_params = trial.params

In [19]:
best_params

{'learning_rate': 3.354100053227975e-05,
 'weight_decay': 0.025573937515468943,
 'dropout_rate': 0.22975851341070952,
 'hidden_size': 317,
 'num_epochs': 9,
 'batch_size': 32,
 'warmup_ratio': 0.11351812674740831,
 'label_smoothing': 0.06721970339723499,
 'gradient_accumulation_steps': 3,
 'max_grad_norm': 0.7154661794845409}

## 7. 모델 저장 및 로드

In [ ]:
# 모델 저장
trainer.save_model('./news_bias_model')  # 저장 경로 변경
tokenizer.save_pretrained('./news_bias_model')  # 저장 경로 변경

# 모델 로드
def load_model(model_path):
    model = NewsBiasModel('klue/roberta-base')  # 모델 클래스 변경
    model.load_state_dict(torch.load(f'{model_path}/pytorch_model.bin'))
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

## 8. 새로운 기사에 대한 예측

In [ ]:
def predict_news(text, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        party_pred = torch.argmax(outputs['party_logits'], dim=1).item()  # stance_logits를 party_logits로 변경

    # 정당 레이블 매핑
    party_mapping = {0: '국민의힘', 1: '민주당', 2: '그외'}
    return party_mapping[party_pred]  # 숫자 레이블을 정당 이름으로 변환하여 반환

In [ ]:
def predict_csv_file(csv_path, model, tokenizer, device):
    # CSV 파일 로드
    df = pd.read_csv(csv_path)

    # 예측 결과를 저장할 리스트
    stance_predictions = []
    sentiment_predictions = []

    # 각 텍스트에 대해 예측 수행
    for text in tqdm(df['text'], desc="Predicting"):
        stance_pred, sentiment_pred = predict_news(text, model, tokenizer, device)
        stance_predictions.append(stance_pred)
        sentiment_predictions.append(sentiment_pred)

    # 예측 결과를 DataFrame에 추가
    df['stance_label'] = stance_predictions
    df['sentiment_label'] = sentiment_predictions

    # 예측 결과를 숫자에서 텍스트로 변환
    stance_mapping = {v: k for k, v in stance_mapping.items()}
    sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}

    df['stance'] = df['stance_label'].map(stance_mapping)
    df['sentiment'] = df['sentiment_label'].map(sentiment_mapping)

    # 결과 저장
    output_path = csv_path.replace('.csv', '_predicted.csv')
    df.to_csv(output_path, index=False)

    # 예측 결과 통계 출력
    print(f"\n{csv_path} 예측 결과:")
    print("\n스탠스 예측 분포:")
    print(df['stance'].value_counts())
    print("\n감성 예측 분포:")
    print(df['sentiment'].value_counts())

    return df

In [ ]:
# 모델 로드
model, tokenizer = load_model('./news_bias_model')  # 모델 경로 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# CSV 파일 예측
csv_path = "/content/drive/MyDrive/텍스트데이터분석을 위한 딥러닝/팀프로젝트/data/전체1.csv"  # 여기에 실제 CSV 파일 경로를 입력하시면 됩니다
predicted_df = predict_csv_file(csv_path, model, tokenizer, device)
predicted_df.head()

In [ ]:
# 저장
predicted_df.to_csv(csv_path, index=False)